# Using Programmable Graphics Shaders

Most planetarium space engines allow users to create custom content by writing their own graphics shaders. This is a powerful way to create highly performant custom content, and to incoroprate more advance visualization techniques into the existing software. Unfortunately this ability is often poorly documented, and poorly exposed. Consequentally very few members of the planetarium community are currently writing their own shaders.

## An example with Uniview: gravitational wave conceptual animation

Uniview does an excellent of exposing the power of shaders to its users through the Uniview Shader Effect System (USES). Documentation for that system can be found [here](href=https://www.diva-portal.org/smash/get/diva2:623127/FULLTEXT01.pdf). This document describes both how to access system variables (such as the simulation time) and how to define new variables that can be controlled inside Uniview's GUI.

For this example we will make a conceptual animation that describes how a gravitational wave distorts space. We will create a cylinder of connected points that are distorted by the gravitational wave. Here is a screenshot of the visualization (equirectangular full sphere view).
![360 image of GWcatepillar](https://github.com/IPSScienceVisualization/Workshops/blob/master/Tokyo2017/data/gwequi5.png?raw=true)

#### How the shaders work

In this example we will pass through the data two times. On the first pass we will draw the fuzzy halos, and on the second pass we will draw the connecting lines. Most of the woks in this visualization is done by the geometry shaders. 

Geometry shaders allow you to convert one piece of geometry into another piece of geometry. In this case we will be 'tricking' the geometry shader, we will load in what it thinks are a series of triangles, but in fact each triangle is data encoded inside inside the nine numbers that are needed to define the triangle.

Uniview allows you to load geometry as and ASCII table with nine values on each line
> x1 y1 z1 x2 y2 z2 x3 y3 z3

For our example we will create a file where each line describes the position of each vertex on the cylinder. There are many possible ways to do this, we just picked one.

In [1]:
# Creating the Uniview data file
nring = 36
ntube = 25
dTheta = 360./nring
dl = 1.0
writefile=open('GWcatepillar/tube.raw',"w")
for i in range(nring):
    for j in range (-ntube,ntube+1):
        writefile.write("{} {} {} {} {} {} {} {} {}\n".format(i*dTheta,j*dl,i,j,nring,ntube,(i+1)*dTheta,min(j+1,ntube-1)*dl,"1"))
writefile.flush()
writefile.close()

##### The key shaders are:
* [GWcatepillar_halos.glsl](https://github.com/IPSScienceVisualization/Workshops/blob/master/Tokyo2017/GWcatepillar/shaders/GWcatepillar_halos.glsl): This geometry shader draws a camerfacing square at the position of each vertex of the cylinder. The cylinder is distorted due to the gravitational wave (using the simulation time to track the wave propagation). A passed variable (length) determines how much of the cylinder is drawn. A color is passed to the fragment shader indicating how much space is distorted at that point.
* [marker_fragment.glsl](https://github.com/IPSScienceVisualization/Workshops/blob/master/Tokyo2017/GWcatepillar/shaders/marker_fragment.glsl): This fragment shader draws one of several marker fragments on its quad based on a passed variable. The default marker is a fuzzy blob. The color of the marker is also determined by a variable passed from the geometry shader.
* [GWcatepillar_lines.glsl](https://github.com/IPSScienceVisualization/Workshops/blob/master/Tokyo2017/GWcatepillar/shaders/GWcatepillar_lines.glsl): This geometry shader draws lines connecting the designated vertex to two adjacent vertices in the cylinder. The cylinder is distorted due to the gravitational wave (using the simulation time to track the wave propagation). A passed variable (length) determines how much of the cylinder is drawn. A color is passed to the fragment shader. 

## Extend this example

Right now this example illustrated only one of the two possible gravitational wave polarizations. Can you extend the example to include the other polarization?